|  Sunrise logo | ![EEW logo](https://github.com/edgi-govdata-archiving/EEW-Image-Assets/blob/master/Jupyter%20instructions/eew.jpg?raw=true) | ![EDGI logo](https://github.com/edgi-govdata-archiving/EEW-Image-Assets/blob/master/Jupyter%20instructions/edgi.png?raw=true) |
|---|---|---|

#### This notebook is licensed under GPL 3.0. Please visit our Github repo for more information: https://github.com/edgi-govdata-archiving/ECHO-COVID19
#### The notebook was collaboratively authored by EDGI following our authorship protocol: https://docs.google.com/document/d/1CtDN5ZZ4Zv70fHiBTmWkDJ9mswEipX6eCYrwicP66Xw/
#### For more information about this project, visit https://www.environmentalenforcementwatch.org/

## How to Run
* A "cell" in a Jupyter notebook is a block of code performing a set of actions making available or using specific data.  The notebook works by running one cell after another, as the notebook user selects offered options.
* If you click on a gray **code** cell, a little “play button” arrow appears on the left. If you click the play button, it will run the code in that cell (“**running** a cell”). The button will animate. When the animation stops, the cell has finished running.
![Where to click to run the cell](https://github.com/edgi-govdata-archiving/EEW-Image-Assets/blob/master/Jupyter%20instructions/pressplay.JPG?raw=true)
* You may get a warning that the notebook was not authored by Google. We know, we authored them! It’s okay. Click “Run Anyway” to continue. 
![Error Message](https://github.com/edgi-govdata-archiving/EEW-Image-Assets/blob/master/Jupyter%20instructions/warning-message.JPG?raw=true)
* **It is important to run cells in order because they depend on each other.**
* Run all of the cells in a Notebook to make a complete report. Please feel free to look at and **learn about each result as you create it**!

# **Let's begin!** 
These first two cells give us access to some external Python code we will need. Hover over the "[ ]" on the top left corner of the cell below and you should see a "play" button appear. Click on it to run the cell then move to the next one.
### 1.  Bring in some code that is stored in a Github project.

In [ ]:
!git clone -b sunrise --single-branch https://github.com/ericnost/ECHO_modules.git
!git clone https://github.com/edgi-govdata-archiving/ECHO-Geo.git
!git clone -b first-draft --single-branch  https://github.com/edgi-govdata-archiving/ECHO-Sunrise.git # This has the utilities file for mapping

### 2.  Run some external Python modules.

In [ ]:
# Import code libraries
%run ECHO_modules/DataSet.py
%run ECHO-Sunrise/utilities.py
import urllib.parse
import pandas as pd
!pip install geopandas &>/dev/null;
!apt install libspatialindex-dev &>/dev/null;
!pip install rtree &>/dev/null;
import geopandas
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import requests
import csv
import datetime
import ipywidgets as widgets

### 3. What facilities does EPA track in Mass?
This may take just a little bit of time to load - there are thousands!

In [ ]:
echo_data_sql = "select * from ECHO_EXPORTER where FAC_STATE = 'MA' and FAC_ACTIVE_FLAG='Y'"

try:
    print(echo_data_sql)
    echo_data = get_data( echo_data_sql, 'REGISTRY_ID' )
    num_facilities = echo_data.shape[0]
    print("\nThere are %s facilities in Massachussets currently tracked in the ECHO database." %(num_facilities))
except pd.errors.EmptyDataError:
    print("\nThere are no facilities in this region.\n")

In [ ]:
#mapper_marker(echo_data)
echo_data

### 4.  Run this next cell to create to choose how you want to *zoom in*: what specific programs you want to look at and whether you want to view this information by county, congressional district or zip code.
Here's where you can learn more about the different programs...

In [ ]:
%run ECHO_modules/make_data_sets.py

# Only list the data set if it has the correct flag set.
data_set_choices = []
for k, v in data_sets.items():
    if ( v.has_echo_flag( echo_data ) ):
        data_set_choices.append( k )

data_set_widget=widgets.Dropdown(
    options=list(data_set_choices),
    description='Data sets:',
    disabled=False,
    value='Greenhouse Gases'
) 
display(data_set_widget)

region_field = { 
    'Congressional District': { "field": "cd" },
    'County': { "field": "county" },
    'State Districts': { "field": "state_districts" },
    'Town': {"field": "town"},
    'Watershed': {"field": "watershed"},
    'Zip Code': { "field": "zip" },
}

style = {'description_width': 'initial'}
select_region_widget = widgets.Dropdown(
    options=region_field.keys(),
    style=style,
    value='State Districts',
    description='Region of interest:',
    disabled=False
)
display( select_region_widget )

### 5. Here are all the facilities in this program
This may take some time because we're looking at all incidents under this program for all facilities across the state!

First, let's get all the data from the database.

In [ ]:
program = data_sets[ data_set_widget.value ]
program_data = None

my_prog_data=get_program_data(echo_data, program, program_data)
my_prog_data=pd.DataFrame(my_prog_data)
my_prog_data

And map it:

In [ ]:
fac = my_prog_data.drop_duplicates(subset=["Index"])
map_of_facilities = mapper_marker(fac)
map_of_facilities

### 6. Here are the geographies we're going to summarize this information at

In [ ]:
# read in and map geojson for the selected geography
geo = region_field[select_region_widget.value]["field"].lower()
geo_json_data = geopandas.read_file("ECHO-Geo/ma_"+geo+".geojson")

m = folium.Map(
    #tiles='Mapbox Bright',
)
folium.GeoJson(
    geo_json_data,
).add_to(m)

bounds = m.get_bounds()
m.fit_bounds(bounds)

m

### 7. Now we bring the geographic data and the facility data together. First, let's rank each geography.

In [ ]:
# first, spatialize my_prog_data
gdf = geopandas.GeoDataFrame(
    my_prog_data, geometry=geopandas.points_from_xy(my_prog_data["FAC_LONG"], my_prog_data["FAC_LAT"]))

join = geopandas.sjoin(gdf, geo_json_data, how="inner", op='intersects')

join.to_csv("full_program_data.csv")

# get geo and attribute data column names
geo_column = {"county": "COUNTY", "state_districts": "REP_DIST"} # EXPAND
att_column = {"Greenhouse Gases": {"col":"ANNUAL_EMISSION", "agg":"sum"},
              "Air Inspections": {"col": "ACTIVITY_TYPE_CODE", "agg": "count"},
              "Clean Water Inspections": {"col":"ACTIVITY_TYPE_CODE", "agg":"count"}} # EXPAND
g = geo_column[geo]
a = att_column[program.name]["col"]

data = join.groupby(join[g])[[a]].agg(att_column[program.name]["agg"])
data.sort_values(by=a, ascending = False)
data.to_csv("geos_ranked.csv")
data

### 8. Now, let's map it!

In [ ]:
data.reset_index(inplace=True)
att_data = data.rename(columns={g: "geo", a: "value"}) 
mp = mapper_area(geo_json_data, att_data, g)
mp

### 9. Rank individual facilities

In [ ]:
ranked = my_prog_data.groupby(["Index", "FAC_NAME", "FAC_LAT", "FAC_LONG"])[[a]].agg(att_column[program.name]["agg"])
ranked.reset_index(inplace=True)
ranked = ranked.set_index("Index")
ranked.sort_values(by=a, ascending=False)
ranked.to_csv("facilities_ranked.csv")
ranked

### 10. Map individual facilities

In [ ]:
ranked['quantile'] = pd.qcut(ranked[a], 4, labels=False, duplicates="drop")
mp = mapper_circle(ranked, a)
mp